In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT_pi_SICK_fs5"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# DEV de SICK

In [2]:
import pandas as pd

In [3]:
df =pd.read_pickle("../predictions/RL_SICK_DEV_with_prediction.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,predictions
0,-1,0.200000,0.4,0.333333,0.845154,0.233221,4.727097,1.000000,1.000000,0,0,0.904762,0,0,0.400000,CONTRADICTION,0
1,2,0.800000,0.0,0.250000,0.200000,0.057993,3.973408,0.531105,0.666667,3,0,0.466667,0,0,0.200000,NEUTRAL,2
2,2,0.500000,0.0,0.500000,0.288675,0.117675,4.273848,0.669376,0.666667,1,0,0.527778,0,0,0.500000,NEUTRAL,2
3,2,1.000000,0.0,0.000000,0.000000,0.136313,3.621765,0.295252,0.400000,2,0,0.361111,0,0,0.000000,NEUTRAL,2
4,1,0.000000,0.0,1.000000,1.000000,0.241997,4.697862,1.000000,1.000000,0,0,1.000000,0,0,1.000000,NEUTRAL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2,1.000000,0.0,0.000000,0.000000,0.067240,3.754545,0.198049,0.333333,1,0,0.361111,0,0,0.000000,NEUTRAL,2
491,2,0.333333,0.0,0.666667,0.577350,0.275896,4.461859,0.732893,1.000000,1,0,0.638889,0,0,0.666667,NEUTRAL,1
492,2,1.000000,0.0,0.000000,0.000000,0.000000,3.783412,0.098616,0.000000,0,0,0.000000,0,0,0.000000,NEUTRAL,2
493,2,1.000000,0.0,0.000000,0.000000,0.028540,3.768143,0.075811,0.500000,2,0,0.333333,0,0,0.000000,NEUTRAL,2


In [4]:
original =pd.read_csv("../../RIT_ollama/SICK/SICK_DEV.csv")
original

,sentence_A,sentence_B,entailment_label
0,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION
1,A person in a black jacket is doing tricks on ...,A skilled person is riding a bicycle on one wheel,NEUTRAL
2,Four children are doing backbends in the gym,Four girls are doing backbends and playing out...,NEUTRAL
3,A player is throwing the ball,Two teams are competing in a football match,NEUTRAL
4,Five children are standing in front of a woode...,Five children are standing in a wooden hut,NEUTRAL
...,...,...,...
490,A biker is riding away from a fence,A man is dancing on the road,NEUTRAL
491,A woman is playing an electric guitar,A kid is playing guitar,NEUTRAL
492,The animal is grazing on the grass,The cop is sitting on a police bike,NEUTRAL
493,The dog is snapping at some droplets of water,A girl in a band is playing an instrument,NEUTRAL


In [5]:
df["sentence_A"]=original["sentence_A"]
df["sentence_B"]=original["sentence_B"]

In [6]:
lista_respuestasOllama=[]

In [7]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'predictions', 'sentence_A', 'sentence_B'],
      dtype='object')

In [8]:
faltantes=[]

# RIT pi fs 5

In [9]:
for i in range(1):
    for index,strings in df[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+''' 
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_pifs5_SICK_DEV_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*500,(i+1)*500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
lista_respuestasOllama

['{\n  "Answer": "CONTRADICTION",\n  "Score of Lexical Semantics (0-1)": "0.6",\n  "Score of Predicate-Argument Structure  (0-1)": "0.8",\n  "Score of Logic (0-1)": "0.7",\n  "Score of Knowledge & Common Sense (0-1)": "0.5"\n}',
 '{\n  "Answer": "Neutral",\n  "Score of Lexical Semantics (0-1)": "0.8",\n  "Score of Predicate-Argument Structure  (0-1)": "0.2",\n  "Score of Logic (0-1)": "0.6",\n  "Score of Knowledge & Common Sense (0-1)": "0.4"\n}',
 '{\n  "Answer": "Implication",\n  "Score of Lexical Semantics (0-1)": "0.2",\n  "Score of Predicate-Argument Structure  (0-1)": "0.3",\n  "Score of Logic (0-1)": "0.4",\n  "Score of Knowledge & Common Sense (0-1)": "0.5"\n}',
 '{\n  "Answer": "Neutral",\n  "Score of Lexical Semantics (0-1)": "0.7",\n  "Score of Predicate-Argument Structure  (0-1)": "0.6",\n  "Score of Logic (0-1)": "0.4",\n  "Score of Knowledge & Common Sense (0-1)": "0.3"\n}',
 '{\n  "Answer": "Implication",\n  "Score of Lexical Semantics (0-1)": "0.2",\n  "Score of Predica